In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
inspector = inspect(engine)

# reflect the tables
inspector.get_table_names()

In [ ]:
# View all of the classes that automap found
measurement_col = inspector.get_columns('measurement')
station_col = inspector.get_columns('station')
for m in measurement_col:
    print(m['name'], m['type'])

In [ ]:
for s in station_col:
    print(s['name'], s['type'])

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
session = Session(engine)

In [ ]:
# Create our session (link) from Python to the DB

measurement_stmt = session.query(measurement).\
    order_by(measurement.id).statement
measurement_df = pd.read_sql_query(measurement_stmt, session.bind)
measurement_df

In [ ]:
count_measurement = measurement_df['station'].nunique()
count_measurement

In [ ]:
station_stmt = session.query(station).\
    order_by(station.id).statement
station_df = pd.read_sql_query(station_stmt, session.bind)
station_df

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
recent_measurement = session.query(measurement).\
    order_by(measurement.date.desc()).statement
recent_measure_df = pd.read_sql_query(recent_measurement, session.bind)
recent_measure_df

In [ ]:
session.query(measurement.date).order_by(measurement.date.desc()).first()

In [ ]:
year_ago = dt.date(2017,8,23) - dt.timedelta(days=365)
year_ago

In [ ]:
twelve_measure = recent_measure_df.loc[recent_measure_df['date'] > '2016-08-22']
twelve_measure_clean = twelve_measure[['date','prcp']]
twelve_measure_clean.sort_values('date', ascending=False)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
twelve_measure = recent_measure_df.loc[recent_measure_df['date'] > '2016-08-22']


# Perform a query to retrieve the data and precipitation scores
twelve_measure_clean = twelve_measure[['date','prcp']]


# Save the query results as a Pandas DataFrame and set the index to the date column
max_twelve = twelve_measure_clean.groupby('date')
max_twelve_df = max_twelve.max()

avg_twelve = twelve_measure_clean.groupby('date')
avg_twelve_df = avg_twelve.mean()

sum_twelve = twelve_measure_clean.groupby('date')
sum_twelve_df = sum_twelve.sum()
# Sort the dataframe by date


# Use Pandas Plotting with Matplotlib to plot the data


sum_twelve_df

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
clean_twelve = twelve_measure_clean.dropna()
sort_clean = clean_twelve.sort_values('date', ascending=True).reset_index()
sort_clean

In [ ]:
new_clean = sort_clean[['date', 'prcp']]
new_clean_two = new_clean.rename(columns={'prcp':'precipitation'})

new_df = new_clean_two.set_index('date')

In [ ]:
plot_twelve = new_df.plot(figsize=(15,10))
plt.xticks(rotation=90)
plt.title('Precipitation vs. Time')
plt.ylabel("Inches")
plt.grid(True)
plt.show()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
recent_station = session.query(station).\
    order_by(station.id).statement
recent_station_df = pd.read_sql_query(recent_station, session.bind)
recent_station_df

In [ ]:
station_sel = [func.count(station.station)]

count_station = session.query(*station_sel).\
    order_by(station.station).all()
count_station

In [ ]:
combined_station = pd.merge(measurement_df, station_df, on='station', how='left')
combined_station

In [ ]:
name = [ 'UPPER WAHIAWA 874.3, HI US', 'PEARL CITY, HI US', 'HONOLULU OBSERVATORY 702.2, HI US','KUALOA RANCH HEADQUARTERS 886.9, HI US',
        'MANOA LYON ARBO 785.2, HI US', 'WAIMANALO EXPERIMENTAL FARM, HI US', 'KANEOHE 838.1, HI US', 'WAIKIKI 717.2, HW US',
        'WAIHEE 837.5, HI US']

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_count = combined_station.groupby('station')
station_count_df = station_count.count()
clean_comb = station_count_df[['id_x']]
clean_comb_station = clean_comb.rename(columns={'id_x':'Station Count'}).sort_values('Station Count')
clean_comb_station['Name'] = name
clean_comb_station

In [ ]:
full_station_sel = [measurement.station,
                    func.count(measurement.station)]

full_count_station = session.query(*full_station_sel).\
    group_by(measurement.station).\
    order_by(func.count(measurement.station)).all()
full_count_station

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.

sel = [measurement.station,
       func.min(measurement.tobs),
       func.max(measurement.tobs),
       func.avg(measurement.tobs)]

active_station = session.query(*sel).\
    filter(measurement.station =='USC00519281').\
    order_by(measurement.station).all()
active_station

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
recent_measurement = session.query(measurement.station, measurement.date, measurement.tobs).\
    filter(measurement.station =='USC00519281').\
    filter(measurement.date > '2016-08-22').statement
recent_measurement_df = pd.read_sql_query(recent_measurement, session.bind)
recent_measurement_df

In [ ]:
active_df = recent_measurement_df[['tobs']]
active_df.plot.hist(bins=12)
plt.grid(True)
plt.show()

# Close session

In [ ]:
# Close Session
session.close()